In [1]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit #for data preprocessing and crass validating 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LogisticRegression #logistic Regression
from sklearn.ensemble import RandomForestRegressor #Random Forest 

from statistics import mean
from hyperopt import Trials, hp, fmin, tpe, STATUS_OK, space_eval #for hyperparameter tuning and minimizing

from cyclic_boosting.pipelines import pipeline_CBClassifier
from sklearn.ensemble import HistGradientBoostingRegressor

from datetime import date
from datetime import datetime

import tensorflow as tf

import keras
import keras.layers as layers
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError

import itertools
from keras.layers import LSTM

import seaborn as sns 

from scipy.stats import boxcox 
from scipy.special import inv_boxcox

from termcolor import colored

import joblib

In [2]:
samples = pd.read_csv('preprocessed_lstm.csv')
samples['date'] = pd.to_datetime(samples['date'])
reading_types = pd.read_csv('reading_types.csv')
# samples.info()

df_lst = [(k, v) for k, v in samples.groupby('building_id')]

models = [[]]


In [3]:
from matplotlib import pyplot as plt
from sklearn.discriminant_analysis import StandardScaler
value_type_ids = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
modelDict = {}
for building, df in df_lst:
    modelDict[building] = []
    for typeId in value_type_ids:
        print(building, '=' * 80)
        train_dates = pd.to_datetime(df['date'])
        multivariate = df.drop(['Unnamed: 0', 'building_id', 'date'], axis = 1)  
        # multivariate.info()
        multivariate = multivariate.astype('float32')
        multivariate = multivariate[[typeId]]

        # boxcox
        # multivariate[typeId] = multivariate[typeId].apply(lambda x: x + 1e-6)
        # df_boxcox_vals, fitted_lambda = boxcox(multivariate[typeId])
        # multivariate[typeId] = df_boxcox_vals

        sz = len(multivariate)
        train_sz = int(sz * 0.9)
        test_sz = len(multivariate) - train_sz

        scaler = StandardScaler()  
        scaler = scaler.fit(multivariate) 
        df_scaled = scaler.transform(multivariate)

        train, test = df_scaled[0:train_sz,:], df_scaled[train_sz:sz,:]
        test_dates = train_dates[train_sz:sz]

        def create_dataset(dataset, look_back = 1):
            dataX, dataY = [], []
            for i in range(len(dataset) - look_back - 1):
                a = dataset[i:(i + look_back), :]
                dataX.append(a)
                dataY.append(dataset[i + look_back, :])
            return np.array(dataX), np.array(dataY)
        
        look_back = 10
        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        
        # reshape input to be [samples, time steps, features]
        trainX, trainY = np.array(trainX), np.array(trainY)
        testX, testY = np.array(testX), np.array(testY)

        # print('trainX shape == {}.'.format(trainX.shape))
        # print('trainY shape == {}.'.format(trainY.shape)) 

        # x_train, x_test, y_train, y_test = train_test_split(df.drop(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], axis = 1), df[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']])
        # x_train = np.asarray(x_train).astype(np.float32)
        # y_train = np.asarray(y_train).astype(np.float32)
        model = keras.Sequential()
        model.add(layers.LSTM(150,input_shape = (trainX.shape[1], trainX.shape[2]), return_sequences = True))
        model.add(LSTM(50, return_sequences = False))
        # model.add(LSTM(100))
        model.add(layers.BatchNormalization())
        model.add(layers.Dense(1, activation= 'linear'))


        early_stop = EarlyStopping(monitor = 'val_loss', patience = 6)

        model.compile(optimizer= tf.keras.optimizers.Adam(), loss= tf.keras.losses.Huber())

        history = model.fit(trainX, trainY, batch_size=64, validation_split = 0.2, epochs=20, verbose = 1, callbacks = [early_stop])
        
        # plt.figure(figsize = (18, 10))
        # plt.plot(history.history['loss'], label='Training loss')
        # plt.plot(history.history['val_loss'], label='Validation loss')
        # plt.legend()
        # plt.show()
        # plt.clf()
        # plt.cla()
        # plt.close()

        predict_range = 24
        test_dates = test_dates[0:predict_range]

        n_future = pd.date_range(test_dates.iloc[0], periods = predict_range, freq = 'H').tolist()

        prediction = model.predict(trainX[-predict_range:])

        # col = 3

        y_pred_future = scaler.inverse_transform(prediction)[:, 0]
        # print(y_pred_future)

        #inverse boxcox
        # y_pred_future = inv_boxcox(y_pred_future, fitted_lambda)
        # y_pred_future = np.vectorize(lambda x: x - 1e-6)(y_pred_future)
        # # y_pred_future = y_pred_future[0] - 1e-6

        forecast_dates = []
        for time_i in n_future:
            forecast_dates.append(time_i)


        df_forecast = pd.DataFrame({'date':np.array(forecast_dates), reading_types.at[int(typeId) - 1, 'reading_type_name']:y_pred_future})
        df_forecast['date']=pd.to_datetime(df_forecast['date'])

        original = df[['date', typeId]]
        original['date']=pd.to_datetime(original['date'])
        original = original[train_sz:train_sz + predict_range]

        # print(reading_types.at[int(typeId) - 1, 'reading_type_name'], '='*100)

        # # print(trainX[-predict_range:])
        # # print(df[train_sz-predict_range:train_sz])
        # # print(df_forecast)
        # # print(original)
        # # print("original", '-'*80)
        # # print(original.head(24))

        # # print("df_forecast", '-'*80)
        # # print(df_forecast.head(24))

        # # original.set_index('date')
        # # df_forecast.set_index('date')

        # print(colored("MEAN SQUARED ERROR: ", 'red'), mean_squared_error(original[typeId], df_forecast[reading_types.at[int(typeId) - 1, 'reading_type_name']]))

        # # plt.figure(figsize=(18,8))
        # # plt.plot(original[typeId],label = "original")
        # # plt.plot(df_forecast[reading_types.at[int(typeId) - 1, 'reading_type_name']],label = "predicted")
        # # plt.title("Time Series Forecast")
        # # plt.xlabel("Date")
        # # plt.ylabel(reading_types.at[int(typeId) - 1, 'reading_type_name'])
        # # plt.legend()
        # # plt.show()

        # sns.lineplot(data= original, x = 'date', y = typeId)
        # sns.lineplot(data = df_forecast, x = 'date', y =  reading_types.at[int(typeId) - 1, 'reading_type_name'])
        # plt.show()
        modelDict[building].append(model)

1 ================================================================================
Epoch 1/20
93/93 [==============================] - 8s 34ms/step - loss: 0.2396 - val_loss: 0.2988
Epoch 2/20
93/93 [==============================] - 2s 23ms/step - loss: 0.1880 - val_loss: 0.2926
Epoch 3/20
93/93 [==============================] - 2s 23ms/step - loss: 0.1792 - val_loss: 0.2796
Epoch 4/20
93/93 [==============================] - 2s 23ms/step - loss: 0.1781 - val_loss: 0.2506
Epoch 5/20
93/93 [==============================] - 2s 23ms/step - loss: 0.1738 - val_loss: 0.2202
Epoch 6/20
93/93 [==============================] - 2s 23ms/step - loss: 0.1780 - val_loss: 0.2000
Epoch 7/20
93/93 [==============================] - 2s 23ms/step - loss: 0.1789 - val_loss: 0.1622
Epoch 8/20
93/93 [==============================] - 2s 23ms/step - loss: 0.1742 - val_loss: 0.1518
Epoch 9/20
93/93 [==============================] - 2s 23ms/step - loss: 0.1745 - val_loss: 0.1380
Epoch 10/20
93/93 [=======

IndexError: single positional indexer is out-of-bounds

In [ ]:
joblib.dump(modelDict, 'modelDict.pkl', compress = 9)

['modelDict.pkl']